In [1]:
from datasets import load_dataset
imdb = load_dataset("imdb")

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating unsupervised split: 100%|███████████████████████████████████████| 50000/50000 [00:00<00:00, 598415.19 examples/s]


In [6]:
imdb["train"]["text"][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

Counter({0: 12500, 1: 12500})

### EDA

In [17]:
# Label distribution
from collections import Counter
training_label_count = Counter(imdb["train"]["label"])
test_label_count = Counter(imdb["test"]["label"])
print(f"training_label_count = {training_label_count}; test_label_count = {test_label_count}")

training_label_count = Counter({0: 12500, 1: 12500}); test_label_count = Counter({0: 12500, 1: 12500})


In [61]:
training_data_tokenized = imdb["train"].map(lambda examples: tokenizer(examples['text']), batched=True)
test_data_tokenized = imdb["test"].map(lambda examples: tokenizer(examples['text']), batched=True)

In [140]:
def tokenize_dataset_to_dataloader(data, shuffle: bool = False, batch_size: int = 32):
    # 1. tokenize
    dataset = data.map(
        lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), 
        batched=True
    )
    # convert to pytorch
    dataset.set_format(type='torch', columns=['input_ids', 'label'])
    # convert to dataloader
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [141]:
# Create data loaders for our datasets; shuffle for imdb["train"], not for validation
training_loader = tokenize_dataset_to_dataloader(imdb["train"], shuffle=True)
test_loader = tokenize_dataset_to_dataloader(imdb["test"], shuffle=False)

Map: 100%|███████████████████████████████████████████████████████████████████| 25000/25000 [00:06<00:00, 3718.54 examples/s]


In [142]:
next(iter(training_loader))

{'label': tensor([1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
         0, 1, 1, 0, 0, 1, 0, 0]),
 'input_ids': tensor([[ 101, 1045, 3342,  ...,    0,    0,    0],
         [ 101, 2023, 2143,  ...,    0,    0,    0],
         [ 101, 4419, 1005,  ...,    0,    0,    0],
         ...,
         [ 101, 2023, 2220,  ...,    0,    0,    0],
         [ 101, 2023, 4038,  ...,    0,    0,    0],
         [ 101, 1045, 2387,  ...,    0,    0,    0]])}

### Model

In [55]:
from transformers import AutoTokenizer, AutoConfig

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)

In [56]:
sample_input_tokens = tokenizer(imdb["train"]["text"][0])
sample_input_tokens

{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 383

In [57]:
config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [158]:
import torch
from torch import nn
import torch.nn.functional as F
from math import sqrt

class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        # token embeddings
        self.token_embeddings = nn.Embedding(
            num_embeddings = config.vocab_size, 
            embedding_dim = config.hidden_size
        )
        # positional embeddings that maps max positions to embed_dim
        self.positional_embeddings = nn.Embedding(
            num_embeddings = config.max_position_embeddings, 
            embedding_dim = config.hidden_size
        )
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        
    def forward(self, tokens):
        seq_len = tokens.size(-1)
        position_ids = torch.arange(seq_len, dtype = torch.long).unsqueeze(0)
        token_embeddings = self.token_embeddings(tokens)
        position_embeddings = self.positional_embeddings(position_ids)
        
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings
    
def scaled_dot_attention(query, key, value): 
    # compute scaled dot attention, given all tensors
    # query.size() = (batch_size, embed_dim, head_dim)
    dim_k = key.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim = -1)
    return torch.bmm(weights, value)
    
    
class AttentionHead(nn.Module):
    def __init__(self, embed_dim: int, head_dim: int):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim) 
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)
        
    def forward(self, x):
        return scaled_dot_attention(
            self.q(x),
            self.k(x),
            self.v(x)
        )
    
class MultiAttentionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        head_dim = config.hidden_size // config.num_attention_heads
        self.attention_heads = nn.ModuleList([
            AttentionHead(embed_dim = embed_dim, head_dim = head_dim) for _ in range(config.num_hidden_layers) 
        ])
        self.output_layer = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.attention_heads], dim = -1)
        x = self.output_layer(x)
        return x
    
    
class FeedForward(nn.Module):
    def __init__(self, embed_dim: int, inter_dim: int):
        super().__init__()
        # simple FNN with GELU & dropout
        self.inter_layer = nn.Linear(embed_dim, inter_dim)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout()
        self.output_layer = nn.Linear(inter_dim, 1)
        
    def forward(self, x):
        x = self.inter_layer(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        x = torch.sigmoid(x) # convert the output to sigmoid so we get probability
        return x
    

class EncoderClassifier(nn.Module):
    def __init__(self, config):
        super().__init__()
        # initialize the layers
        # 1. custom embedding layer that combines token + positional embeddings
        self.embeddings = Embeddings(config)
        # 2. attention multi heads
        self.attention_heads = MultiAttentionHead(config)
        # 3. FNN with probability output
        self.feedforward = FeedForward(
            embed_dim = config.hidden_size, 
            inter_dim = config.intermediate_size, 
        )
        
    def forward(self, tokens):
        x = self.embeddings(tokens)
        x = self.attention_heads(x)[:, 0, :] # select hidden state of [CLS] token
        x = self.feedforward(x)[:, 0]
        return x

### Training

In [161]:
model = EncoderClassifier(config)
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

running_loss = 0.

for i, data in enumerate(training_loader):
    # Every data instance is an input + label pair
    input_ids = data["input_ids"]
    labels = data["label"].float()
    # Zero your gradients for every batch!
    optimizer.zero_grad()

    # Make predictions for this batch
    outputs = model(input_ids)

    # Compute the loss and its gradients
    loss = loss_fn(outputs, labels)
    loss.backward()

    # Adjust learning weights
    optimizer.step()

    # Gather data and report
    running_loss += loss.item()
    if i % 1000 == 999:
        last_loss = running_loss / 1000 # loss per batch
        print('  batch {} loss: {}'.format(i + 1, last_loss))
        tb_x = epoch_index * len(training_loader) + i + 1
        tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        running_loss = 0.

KeyboardInterrupt: 

### Evaluation